<a href="https://colab.research.google.com/github/tousyou/keras/blob/master/multi_value_feature_column.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import tensorflow as tf
import numpy as np
from collections import OrderedDict


In [83]:
FEATURE_COLUMNS = ['Label']
for i in range(1,14):
  FEATURE_COLUMNS.append('I{0}'.format(i))
FEATURE_COLUMNS.append('S1')
print(FEATURE_COLUMNS)

SELECT_COLUMNS = ['Label']
for i in range(1,14):
  SELECT_COLUMNS.append('I{0}'.format(i))

['Label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13', 'S1']


In [123]:
def _column_to_csv_defaults():
  """parse columns to record_defaults param in tf.decode_csv func
    Return: 
      OrderedDict {'feature name': [''],...}
  """
  csv_defaults = OrderedDict()
  csv_defaults['Label'] = [0]  # first label default, empty if the field is must
  for i in range(1,14):
    csv_defaults['I{0}'.format(i)] = [0]
  csv_defaults['S1'] = ['']
  return csv_defaults

csv_defaults = _column_to_csv_defaults()
csv_defaults
print(csv_defaults.values())

def parser(value):
  """Parse train and eval data with label
    Args:
    value: Tensor("arg0:0", shape=(), dtype=string)
  """
  # `tf.decode_csv` return rank 0 Tensor list: <tf.Tensor 'DecodeCSV:60' shape=() dtype=string>
  # na_value fill with record_defaults
  columns = tf.io.decode_csv(
                value, 
                record_defaults=list(csv_defaults.values()),
                #record_defaults = csv_default_values,
                use_quote_delim=False,
                field_delim='\t')
  features = dict(zip(csv_defaults.keys(), columns))
  for f, tensor in features.items():
    if isinstance(csv_defaults[f][0], str):
      # input must be rank 1, return SparseTensor
      # print(st.values)  # <tf.Tensor 'StringSplit_11:1' shape=(?,) dtype=string>
      #features[f] = tf.compat.v1.string_split([tensor], ',').values  # tensor shape (?,)
      features[f] =  tf.strings.split([tensor], ',').to_tensor()
    else:
      features[f] = tf.expand_dims(tensor, 0)  # change shape from () to (1,)
      
  #labels = tf.equal(features.pop('label'), 1)
  labels = features.pop('Label')
  return features, labels

odict_values([[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], ['']])


In [124]:
"""
ds_train = tf.data.experimental.make_csv_dataset(
    './sample_data/criteo.csv',
    batch_size=1,
    column_names = FEATURE_COLUMNS,
    #select_columns = SELECT_COLUMNS,
    label_name = 'Label',
    field_delim = '\t',
    num_epochs =1
)
"""
ds_train = tf.data.TextLineDataset('./sample_data/criteo.csv').map(lambda ex: parser(ex))

examples, labels = next(iter(ds_train)) # 第一个批次
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 {'I1': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, 'I2': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, 'I3': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([5], dtype=int32)>, 'I4': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, 'I5': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1382], dtype=int32)>, 'I6': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([4], dtype=int32)>, 'I7': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([15], dtype=int32)>, 'I8': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, 'I9': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([181], dtype=int32)>, 'I10': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>, 'I11': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, 'I12': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, 'I13': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, 'S

In [125]:
numerical_columns = []
for i in range(1,13):
  numerical_columns.append(tf.feature_column.numeric_column('I{0}'.format(i)))

category_columns = []
item = tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_hash_bucket('S1',200))
category_columns.append(item)


In [130]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
  tf.keras.layers.DenseFeatures(numerical_columns+category_columns), # Placing the feature into tf.keras.layers.DenseFeatures
  #tf.keras.layers.DenseFeatures(numerical_columns),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['AUC'])

train_data = ds_train.batch(64).repeat(2).shuffle(200)
model.fit(train_data, epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
32/32 [==============================] - 0s 2ms/step - loss: 61.1663 - auc: 0.5262
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 38.7487 - auc: 0.5404
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 18.2265 - auc: 0.5510
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 3.8172 - auc: 0.5580
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 2.7940 - auc: 0.5515
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 1.6482 - auc: 0.5647
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 0.9769 - auc: 0.6163
Epoch 8/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7261 - auc: 0.6469
Epoch 9/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7908 - auc: 0.6494
Epoch 10/10
32/32 [==============================]

In [29]:
def _example(categories):
    f = tf.train.Feature(bytes_list=tf.train.BytesList(value=categories))
    fs = tf.train.Features(feature={'color': f})
    return tf.train.Example(features=fs).SerializeToString()

vocab = ['red', 'green', 'blue']
feature_spec = {'color': tf.io.VarLenFeature(tf.string)}


batch = [_example([b'red', b'green']),
         _example([]),
         _example([b'blue', b'blue', b'yellow']),
         _example(3*[b'red'] + 1*[b'green'])]
"""
batch = [_example([b'red']),
         _example([]),
         _example([b'blue']),
         _example([b'green'])]
"""
batch_parsed = tf.io.parse_example(batch, feature_spec)

#color_feature = tf.feature_column.categorical_column_with_vocabulary_list('color', vocab)
color_feature = tf.feature_column.categorical_column_with_hash_bucket('color',10)
color_feature = tf.feature_column.indicator_column(color_feature)
print(color_feature)

print(batch)
result_tensor = tf.compat.v1.feature_column.input_layer(batch_parsed, [color_feature])
print(result_tensor)


IndicatorColumn(categorical_column=HashedCategoricalColumn(key='color', hash_bucket_size=10, dtype=tf.string))
[b'\n\x19\n\x17\n\x05color\x12\x0e\n\x0c\n\x03red\n\x05green', b'\n\r\n\x0b\n\x05color\x12\x02\n\x00', b'\n!\n\x1f\n\x05color\x12\x16\n\x14\n\x04blue\n\x04blue\n\x06yellow', b'\n#\n!\n\x05color\x12\x18\n\x16\n\x03red\n\x03red\n\x03red\n\x05green']
tf.Tensor(
[[1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 2. 0. 0. 0. 0.]
 [3. 0. 0. 0. 1. 0. 0. 0. 0. 0.]], shape=(4, 10), dtype=float32)
